In [1]:
import numpy as np
import tensorflow as tf
import cv2
import lucid.scratch.web.svelte as lucid_svelte
import lucid.modelzoo.vision_models as models
import lucid.optvis.render as render
import lucid.optvis.objectives as objectives
import os
import time

from lucid.misc.io import show, load
from lucid.misc.io.showing import _image_url
from lucid_load_model import CovidNetB
from data import process_image_file

covidnet = CovidNetB()

In [4]:
def get_average_neuron_firing(layer='conv5_block3_out/add'):
    total = 0
    no = len(os.listdir('data/test'))
    i = 1
    for image_file in os.listdir('data/test'):
        if (i%50 == 0):
            print(str(i)+'/'+str(no))
        i += 1
        processed_img = process_image_file('data/test/' + image_file, 0, 480)

        # Compute the activations
        with tf.Graph().as_default() as graph, tf.Session():
            t_input = tf.placeholder(tf.float32, [480, 480, 3])
            T = render.import_model(covidnet, t_input, t_input)
            acts = T(layer).eval({t_input: processed_img})[0]

        acts = np.mean(acts, axis=(0,1))
        total = acts + total
    average = total/no
    return average

In [5]:
'''
average = get_average_neuron_firing()
print(average)
average_out_file = open('output/lucid/conv5/average.txt', 'w')
np.savetxt(average_out_file, average)
average_out_file.close()
'''

"\naverage = get_average_neuron_firing()\nprint(average)\naverage_out_file = open('output/lucid/conv5/average.txt', 'w')\nnp.savetxt(average_out_file, average)\naverage_out_file.close()\n"

In [6]:
def get_all_neuron_firing(layer='conv5_block3_out/add'):
    image_list = os.listdir('data/test')
    all_acts = np.zeros((len(image_list), 15,15,2048))
    for i in range(len(image_list)):
        processed_img = process_image_file(os.path.join('data/test',image_list[i]), 0, 480)
        if (i%50 == 0):
            print(str(i)+'/'+str(len(image_list)))

        # Compute the activations
        with tf.Graph().as_default() as graph, tf.Session():
            t_input = tf.placeholder(tf.float32, [480, 480, 3])
            T = render.import_model(covidnet, t_input, t_input)
            acts = T(layer).eval({t_input: processed_img})[0]

        all_acts[i] = acts
    return all_acts

In [7]:
all_acts = get_all_neuron_firing()
#np.savetxt('output/lucid/conv5/all_firings.txt',all_acts)
print(np.shape(all_acts))

acts = []
tic = time.perf_counter()
for channel in range(2048):
    channel_activations = []
    if ((channel%50 == 0) or (channel > 2020)):
        toc = time.perf_counter()
        print(str(channel) + '/2048 - ' + str(toc-tic))
        tic = time.perf_counter()
    for row in range(15):
        for col in range(15):
            for pic in range(1535):
                channel_activations.append(-all_acts[pic, row, col, channel])
    channel_activations.sort()
    acts.append(channel_activations)
print(np.shape(acts))

out_file = open('output/lucid/conv5/all_firings2.txt', 'w')
np.savetxt(out_file,acts)
print('saved')
out_file.close()

0/1535



50/1535
100/1535
150/1535
200/1535
250/1535
300/1535
350/1535
400/1535
450/1535
500/1535
550/1535
600/1535
650/1535
700/1535
750/1535
800/1535
850/1535
900/1535
950/1535
1000/1535
1050/1535
1100/1535
1150/1535
1200/1535
1250/1535
1300/1535
1350/1535
1400/1535
1450/1535
1500/1535
(1535, 15, 15, 2048)
0/2048 - 0.0001616
50/2048 - 21.2621279
100/2048 - 21.065711200000003
150/2048 - 21.1882009
200/2048 - 20.557715699999996
250/2048 - 19.8800035
300/2048 - 19.8548495
350/2048 - 20.36456369999999
400/2048 - 21.6495788
450/2048 - 21.419713799999982
500/2048 - 21.600520899999992
550/2048 - 40.46532930000001
600/2048 - 21.517620700000037
650/2048 - 22.04822539999998
700/2048 - 22.018112799999983
750/2048 - 21.804116399999998
800/2048 - 27.422631699999954
850/2048 - 23.326619199999982
900/2048 - 22.1001215
950/2048 - 22.92547399999995
1000/2048 - 22.96444360000004
1050/2048 - 132.46049620000002
1100/2048 - 21.11096729999997
1150/2048 - 21.62492629999997
1200/2048 - 21.308076200000073
1250/2048 -

In [2]:
acts = np.loadtxt('output/lucid/conv5/all_firings2.txt')
print(np.shape(acts))

(2048, 345375)


In [3]:
shape = np.shape(acts)
dist = int(shape[1]/100)
init_dist = shape[1] - 99*dist
print(dist)
print(init_dist)
print(init_dist + 99*dist)



'''
size = np.shape(acts)[1]
dist = size/100
small_acts = []

for channel in range(2048):
    channel_acts = []
    '''

3453
3528
345375


'\nsize = np.shape(acts)[1]\ndist = size/100\nsmall_acts = []\n\nfor channel in range(2048):\n    channel_acts = []\n    '

In [13]:
small_acts = []
for channel in range(2048):
    channel_acts = []
    channel_acts.append(acts[channel][0])
    current_dist = init_dist-1
    channel_acts.append(acts[channel][current_dist])
    for i in range(99):
        current_dist += dist
        channel_acts.append(acts[channel][current_dist])
    small_acts.append(channel_acts)
print(np.shape(small_acts))

(2048, 101)


In [14]:
out_file = open('output/lucid/conv5/firings_distribution.txt', 'w')
np.savetxt(out_file,small_acts)
print('saved')
out_file.close()

saved
